this script is for all mixing, including mixing with background noise and mixup to augment sound

In [1]:
import os
import glob
import soundfile as sf
import pandas as pd

In [2]:
# read seltab_list excel file as reference for SNR value range
seltab_list = pd.read_excel('F:/MSc Ecology & Data Science Research/Metadata/public_domainSNR.xlsx')
seltab_list.describe()

,selec,channel,start,end,bottom.freq,top.freq,orig_start,orig_end,time_gap,group_id,SNR
count,4776.000000,4776.000000,4776.000000,4776.000000,4776.000000,4776.000000,2388.000000,2388.000000,4776.000000,4776.000000,4776.000000
mean,36.989531,1.080402,64.916137,66.700949,0.578165,5.213838,65.476059,66.047450,2.444500,34.059045,9.007968
std,38.207232,0.271943,65.692405,65.713666,0.323115,3.023982,65.683132,65.722189,5.132444,35.595862,7.805112
min,1.000000,1.000000,0.000000,0.316433,0.000000,0.674200,0.000000,0.316433,-0.566045,1.000000,-7.265681
25%,9.000000,1.000000,14.884672,16.637237,0.396575,2.783000,15.519001,16.069445,0.145216,8.000000,2.663105
50%,23.000000,1.000000,44.398365,46.208088,0.531300,4.995807,45.108584,45.743928,0.625062,21.000000,7.325478
75%,54.000000,1.000000,92.473440,94.274035,0.675500,6.505600,93.495261,93.933631,2.323336,50.000000,14.138996
max,189.000000,2.000000,334.841280,337.565847,3.232700,22.050000,334.841280,336.633308,74.532614,177.000000,39.151764


In [3]:
meanSNR = seltab_list['SNR'].mean()
minSNR = seltab_list['SNR'].min()
maxSNR = seltab_list['SNR'].max()

In [ ]:
%%capture 
from audiomentations import AddBackgroundNoise, PolarityInversion

# the code is for dartmoor weak labelled data augmentation
# set up folders
input_for_high  = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_high snr_weak_call"
input_for_low   = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_low snr_weak_call"
noise_for_high  = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_high snr_weak_noise"
noise_for_low   = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_low snr_weak_noise"

output_folder_high = "F:/MSc Ecology & Data Science Research/3. train_data/1. Wilcoxon/dr_highsnr_weak/Red Fox"
output_folder_low  = "F:/MSc Ecology & Data Science Research/3. train_data/1. Wilcoxon/dr_lowsnr_weak/Red Fox"

# create the augmentation transform
# specify the range of SNR, the rest are defaults
# this function will randomly pick SNR within the specified range
transform_low = AddBackgroundNoise(
    sounds_path=noise_for_low,
    min_snr_db=minSNR,
    max_snr_db=meanSNR,
    noise_transform=PolarityInversion(),
    p=1.0
)

transform_high = AddBackgroundNoise(
    sounds_path=noise_for_high,
    min_snr_db=meanSNR,
    max_snr_db=maxSNR,
    noise_transform=PolarityInversion(),
    p=1.0
)



# create a list of wav files in input folder
sound_paths_low = glob.glob(os.path.join(input_for_low, "*.wav"))
sound_paths_high = glob.glob(os.path.join(input_for_high, "*.wav"))


# loop, augment, and save
for sound_path in sound_paths_low:
    # read sound files
    samples, sample_rate = sf.read(sound_path)
    # apply augmentation
    augmented_samples = transform_low(samples, sample_rate)
    # save back out (same filename into output_folder)
    fname = os.path.basename(sound_path)
    out_path = os.path.join(output_folder_low, fname)
    sf.write(out_path, augmented_samples, sample_rate)

# loop, augment, and save
for sound_path in sound_paths_high:
    # read sound files
    samples, sample_rate = sf.read(sound_path)
    # apply augmentation
    augmented_samples = transform_high(samples, sample_rate)
    # save back out (same filename into output_folder)
    fname = os.path.basename(sound_path)
    out_path = os.path.join(output_folder_high, fname)
    sf.write(out_path, augmented_samples, sample_rate)

In [ ]:
%%capture 
from audiomentations import AddBackgroundNoise, PolarityInversion

# the code is for dartmoor strong labelled data augmentation
# set up folders
input_for_high  = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_high snr_strong_call"
input_for_low   = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_low snr_strong_call"
noise_for_high  = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_high snr_strong_noise"
noise_for_low   = "F:/MSc Ecology & Data Science Research/3. augment data_train/Wilcoxon/Dartmoor_low snr_strong_noise"

output_folder_high = "F:/MSc Ecology & Data Science Research/3. train_data/1. Wilcoxon/dr_highsnr_strong/Red Fox"
output_folder_low  = "F:/MSc Ecology & Data Science Research/3. train_data/1. Wilcoxon/dr_lowsnr_strong/Red Fox"

# create the augmentation transform
# specify the range of SNR, the rest are defaults
# this function will randomly pick SNR within the specified range
transform_low = AddBackgroundNoise(
    sounds_path=noise_for_low,
    min_snr_db=minSNR,
    max_snr_db=meanSNR,
    noise_transform=PolarityInversion(),
    p=1.0
)

transform_high = AddBackgroundNoise(
    sounds_path=noise_for_high,
    min_snr_db=meanSNR,
    max_snr_db=maxSNR,
    noise_transform=PolarityInversion(),
    p=1.0
)



# create a list of wav files in input folder
sound_paths_low = glob.glob(os.path.join(input_for_low, "*.wav"))
sound_paths_high = glob.glob(os.path.join(input_for_high, "*.wav"))


# loop, augment, and save
for sound_path in sound_paths_low:
    # read sound files
    samples, sample_rate = sf.read(sound_path)
    # apply augmentation
    augmented_samples = transform_low(samples, sample_rate)
    # save back out (same filename into output_folder)
    fname = os.path.basename(sound_path)
    out_path = os.path.join(output_folder_low, fname)
    sf.write(out_path, augmented_samples, sample_rate)

# loop, augment, and save
for sound_path in sound_paths_high:
    # read sound files
    samples, sample_rate = sf.read(sound_path)
    # apply augmentation
    augmented_samples = transform_high(samples, sample_rate)
    # save back out (same filename into output_folder)
    fname = os.path.basename(sound_path)
    out_path = os.path.join(output_folder_high, fname)
    sf.write(out_path, augmented_samples, sample_rate)

A path to a file/folder with sound(s), or a list of file/folder paths, must be specified. These sounds should ideally be at least as long as the input sounds to be transformed. Otherwise, the background sound will be repeated, which may sound unnatural.
What to do before mixing:
- import xeno canto metadata
- import dartmoor recording metadata
- get the length of xeno canto file 
- crop the background noise of Dartmoor recording so the length would be the same as xeno canto file
- get random dartmoor background noise
- match the background noise with xeno canto file randomly

It works, with caveat however
- if you want to get the SNR that you want, you must use the most clear signal to overlay with the noise
- if the sound already has noise, the SNR would not add linearly
- it will add noise on top of noise and you can only estimates
- so I guess for this case, only use the cleanest sound, overlay with the noise, and modify the SNR through the module